In [1]:
import openai,os,requests,json,re
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

### List up

In [2]:
list_up_format = """\
あなたは{who}です。\
{what}をもれなくリストアップしてください。
"""

In [3]:
who = '管理栄養士'
what = '食品名'
list_up_prompt = list_up_format.format(
    who=who,what=what
)
print(list_up_prompt)

あなたは管理栄養士です。食品名をもれなくリストアップしてください。



In [4]:
messages = [
    {
        'role': 'system',
        "content": list_up_prompt
    }
]

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    # functions=[function],
    # function_call={'name':function['name']}
)
message = response["choices"][0]["message"]

In [6]:
print(message['content'])

以下に、さまざまな食品名の例をリストアップします。

1. りんご
2. ばなな
3. いちご
4. オレンジ
5. すいか
6. みかん
7. レモン
8. ぶどう
9. さくらんぼ
10. パイナップル
11. マンゴー
12. きゅうり
13. にんじん
14. トマト
15. じゃがいも
16. キャベツ
17. レタス
18. ほうれん草
19. ブロッコリー
20. もやし
21. さつまいも
22. ごぼう
23. たけのこ
24. 玉ねぎ
25. にら
26. ねぎ
27. もも
28. さば
29. いわし
30. さんま

これらは、果物や野菜、魚などの一般的な食品名の一部です。ただし、リストはさらに長くなる可能性があります。


In [7]:
function = {
    "name": "get_list",
    "description": "リストを得る",
    "parameters": {
        "type": "object",
        "properties": {
            "list": {
                "type": "array",
                "description": "リスト",
                "items": {
                    "type": "object", "description": "リストの要素",
                    "properties": {
                        "name": {
                            "type": "string", "description": "要素の名称",
                        },
                        "class": {
                            "type": "string", "description": "要素の分類",
                        }
                    }
                }
            }
        },
        "required": ["list"],
    }
}

In [8]:
system_prompt = '入力テキストからすべての要素をリストで出力してください。'
messages = [
    {'role':'system','content':system_prompt},
    {'role':'user','content':message['content']},
]

In [9]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=[function],
    function_call={'name':function['name']}
)
list_up = response["choices"][0]["message"]

In [94]:
data = json.loads(list_up['function_call']['arguments'])['list']

In [95]:
print(data)

[{'name': 'りんご', 'class': '果物'}, {'name': 'ばなな', 'class': '果物'}, {'name': 'いちご', 'class': '果物'}, {'name': 'オレンジ', 'class': '果物'}, {'name': 'すいか', 'class': '果物'}, {'name': 'みかん', 'class': '果物'}, {'name': 'レモン', 'class': '果物'}, {'name': 'ぶどう', 'class': '果物'}, {'name': 'さくらんぼ', 'class': '果物'}, {'name': 'パイナップル', 'class': '果物'}, {'name': 'マンゴー', 'class': '果物'}, {'name': 'きゅうり', 'class': '野菜'}, {'name': 'にんじん', 'class': '野菜'}, {'name': 'トマト', 'class': '野菜'}, {'name': 'じゃがいも', 'class': '野菜'}, {'name': 'キャベツ', 'class': '野菜'}, {'name': 'レタス', 'class': '野菜'}, {'name': 'ほうれん草', 'class': '野菜'}, {'name': 'ブロッコリー', 'class': '野菜'}, {'name': 'もやし', 'class': '野菜'}, {'name': 'さつまいも', 'class': '野菜'}, {'name': 'ごぼう', 'class': '野菜'}, {'name': 'たけのこ', 'class': '野菜'}, {'name': '玉ねぎ', 'class': '野菜'}, {'name': 'にら', 'class': '野菜'}, {'name': 'ねぎ', 'class': '野菜'}, {'name': 'もも', 'class': '魚'}, {'name': 'さば', 'class': '魚'}, {'name': 'いわし', 'class': '魚'}, {'name': 'さんま', 'class': '魚'}]


```json
{
    "name": "str",
    "class": "str",
    "season": {
        "start": "str: YY/MM/DD",
        "end": "str: YY/MM/DD",
    },
    "cooking_methods": ["str"],
    "compatible_ingredients": ["str"],
    "descriptions": {
        "nutrition": "str",
        "delicious_ways_to_eat": "str",
        "season": "str",
    },
}
```

In [137]:
data_format = {
    "type": "object",
    "properties": {
        "name": {
            "type": "string", "description": "食品名。（）は使わない"
        },
        "class": {
            "type": "string", "description": "食品分類",
            "enum": ["野菜","きのこ","肉","魚","加工肉","加工魚","豆製品","卵・乳製品"]
        },
        "season": {
            "type": "object", "description": "旬の時期",
            "properties": {
                "start": {"type": "string", "description": "旬の時期の開始（MM/DD）"},
                "end": {"type": "string", "description": "旬の時期の終了（MM/DD）"},
            }
        },
        "cooking_method": {
            "type": "array", "description": "適した調理方法",
            "items": {
                "type": "string",
                "description": "調理方法",
                "enum": ["焼く","炒める","煮る","ゆでる","蒸す","和える","生"]
            },
        },
        # "description": {
        #     "type": "object", "description": "",
        # },
        "compatible_ingredients": {
            "type": "array", "description": "相性の良い食材リスト",
            "items": {
                "type": "string", "description": "相性の良い食材"
            }
        }
    },
    "required": ['name','class','season','cooking_method','compatible_ingredients']
}

In [138]:
data_extract_format = """\
## Data
{data}

## Components
{components}

## Instruction
[Data]について、[Components]を埋める情報を全て説明してください。
"""

In [139]:
d = data[0]['name']

In [140]:
print(d)

りんご


In [141]:
components = ['食材名','食品分類','旬の時期','適した調理方法','相性の良い食材']
data_extract_prompt = data_extract_format.format(
    data=d, components=','.join(components)
)

In [142]:
# print(data_extract_prompt)

In [143]:
messages = [
    {'role': 'system', 'content': data_extract_prompt}
]

In [144]:
# print(json.dumps(function,indent=2,ensure_ascii=False))

In [145]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    temperature=0.,
    # functions=[function],
    # function_call={'name':function['name']}
)
message = response["choices"][0]["message"]

In [146]:
print(message['content'])

[Components]
- 食材名: りんご
- 食品分類: 果物
- 旬の時期: 秋から冬
- 適した調理方法: 生食、ジャムやジュースの作成、焼きりんご、りんごのパイなどのデザートの材料として使用されることが多い
- 相性の良い食材: シナモン、レモン、バター、クルミ、はちみつ、シナモン、バニラアイスクリームなど


In [147]:
messages = [
    {'role': 'system', 'content': message['content']}
]

In [148]:
function = {
    'name': 'get_food_data',
    'description': '食材データを得る',
    'parameters': data_format
}
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    temperature=0.,
    functions=[function],
    function_call={'name':function['name']}
)
message = response["choices"][0]["message"]

In [149]:
food_data = json.loads(message['function_call']['arguments'])
print(food_data)

{'name': 'りんご', 'class': '果物', 'season': {'start': '秋', 'end': '冬'}, 'cooking_method': ['生'], 'compatible_ingredients': ['シナモン', 'レモン', 'バター', 'クルミ', 'はちみつ', 'シナモン', 'バニラアイスクリーム']}


In [150]:
database = []
for d in data:
    

SyntaxError: incomplete input (3122223509.py, line 3)